In [1]:
import numpy as np
import pandas as pd
import warnings
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
warnings.filterwarnings('ignore')

# Any results you write to the current directory are saved as output.

/kaggle/input/output-data/train_nmf.csv
/kaggle/input/output-data/test_nmf.csv
/kaggle/input/output-data/train_pca.csv
/kaggle/input/output-data/test_pca.csv
/kaggle/input/cs334-project-preprocess/xTrain_renfe_oh.csv
/kaggle/input/cs334-project-preprocess/__results__.html
/kaggle/input/cs334-project-preprocess/yTrain_renfe_oh.csv
/kaggle/input/cs334-project-preprocess/__resultx__.html
/kaggle/input/cs334-project-preprocess/yTest_renfe_oh.csv
/kaggle/input/cs334-project-preprocess/__notebook__.ipynb
/kaggle/input/cs334-project-preprocess/__output__.json
/kaggle/input/cs334-project-preprocess/xTest_renfe_oh.csv
/kaggle/input/cs334-project-preprocess/custom.css


In [2]:
xTrain_oh = pd.read_csv("/kaggle/input/cs334-project-preprocess/xTrain_renfe_oh.csv")
xTest_oh = pd.read_csv("/kaggle/input/cs334-project-preprocess/xTest_renfe_oh.csv")
yTrain_oh = pd.read_csv("/kaggle/input/cs334-project-preprocess/yTrain_renfe_oh.csv")
yTest_oh = pd.read_csv("/kaggle/input/cs334-project-preprocess/yTest_renfe_oh.csv")

In [3]:
from sklearn.decomposition import NMF

def run_nmf(xTrain_oh, xTest_oh):
    approxTol=1
    maxK = np.max(xTrain_oh.shape[1])
    lastErr = np.finfo(float).max
    m = 0
    for k in range(1, maxK, 1):
        nmf = NMF(n_components=k)
        nmf.fit(xTrain_oh)
        err = nmf.reconstruction_err_
        if lastErr - err < approxTol or err < approxTol:
            m = k
            break
        lastErr = err
    print("NMF: number of columns is ", k)
    train_nmf = pd.DataFrame(nmf.transform(xTrain_oh))
    test_nmf = pd.DataFrame(nmf.transform(xTest_oh))
    
    train_nmf.to_csv("train_nmf.csv", index=False)
    test_nmf.to_csv("test_nmf.csv", index=False)
    return train_nmf, test_nmf
    
train_nmf, test_nmf = run_nmf(xTrain_oh, xTest_oh)


NMF: number of columns is  17


In [4]:
# These are commands used for reading the saved dataset. 
#train_nmf = pd.read_csv("../input/output-data/train_nmf.csv")
#test_nmf = pd.read_csv("../input/output-data/test_nmf.csv")

In [5]:
from sklearn.decomposition import PCA
def run_pca(xTrain, xTest):
    # set the shape to be the max
    pca = PCA(n_components=xTrain.shape[1])
    pca.fit(xTrain)
    # calculate number of components to get to 98%
    exp_var = pca.explained_variance_ratio_
    tot_var = np.cumsum(exp_var)
    k = np.argmax(tot_var > 0.98) + 1
    # refit it to this value
    print("PCA: number of PC is ", k)
    pca = PCA(n_components=k)
    train_pca = pca.fit_transform(xTrain)
    test_pca = pca.transform(xTest)
    
    train_pca = pd.DataFrame(train_pca)
    test_pca = pd.DataFrame(test_pca)
    
    train_pca.to_csv("train_pca.csv", index=False)
    test_pca.to_csv("test_pca.csv", index=False)
    
    return train_pca, test_pca

train_pca, test_pca = run_pca(xTrain_oh, xTest_oh)


PCA: number of PC is  1


In [6]:
# These are commands used for reading the saved dataset.
#train_pca = pd.read_csv("../input/output-data/train_pca.csv")
#test_pca = pd.read_csv("../input/output-data/test_pca.csv")

In [7]:
from sklearn.neighbors import KNeighborsRegressor as knn
#from skmultilearn.adapt import MLkNN as knn
def knn_acc(xTrain, yTrain, xTest, yTest):
    
    ###
    param_grid = {'n_neighbors': [1, 3, 5, 7, 9, 11]}
    model = GridSearchCV(knn(), param_grid, cv=5)
    model = model.fit(xTrain, yTrain['price'])
    print("For KNN:")
    print("Best parameters found by grid search:")
    print(model.best_params_)
    print("Best CV score:")
    print(model.best_score_)
    
    preds = model.predict(xTest)
    error = sqrt(mean_squared_error(yTest['price'],preds))
    trainScore = model.score(xTrain, yTrain['price'])
    testScore = model.score(xTest, yTest['price'])
    r2 = r2_score(yTest['price'], preds)

    print()
    print('trainScore of knn: ', trainScore)
    print('testScore of knn: ', testScore)
    print('RMSE of test: ', error)
    print('R-square of test: ', r2)
    print()
    
    
knn_acc(xTrain_oh, yTrain_oh, xTest_oh, yTest_oh)
knn_acc(train_nmf, yTrain_oh, test_nmf, yTest_oh)
knn_acc(train_pca, yTrain_oh, test_pca, yTest_oh)

For KNN:
Best parameters found by grid search:
{'n_neighbors': 7}
Best CV score:
0.9685978204325278

trainScore of knn:  0.9734909417185863
testScore of knn:  0.9701948459114657
RMSE of test:  4.455368625931163
R-square of test:  0.9701948459114655

For KNN:
Best parameters found by grid search:
{'n_neighbors': 5}
Best CV score:
0.9674593287014198

trainScore of knn:  0.9732373469241249
testScore of knn:  0.971207757477892
RMSE of test:  4.379007635744671
R-square of test:  0.9712077574778921

For KNN:
Best parameters found by grid search:
{'n_neighbors': 7}
Best CV score:
0.9689130811244683

trainScore of knn:  0.9732459822013024
testScore of knn:  0.9708967122100443
RMSE of test:  4.4025975089203815
R-square of test:  0.9708967122100443



In [8]:
from sklearn.svm import SVR
def svr_acc(xTrain, yTrain, xTest, yTest):
    svr = SVR()
    svr.fit(xTrain, yTrain)
    preds = svr.predict(xTest)
    error = sqrt(mean_squared_error(yTest['price'],preds))
    r2 = r2_score(yTest['price'], preds)
    
    print('For SVR:')
    print('TrainScore: ', svr.score(xTrain, yTrain['price']))
    print('TestScore: ', svr.score(xTest, yTest['price']))
    print('RMSE of test: ', error)
    print('R-square of test: ', r2)
    print()
    

svr_acc(xTrain_oh, yTrain_oh, xTest_oh, yTest_oh)
svr_acc(train_nmf, yTrain_oh, test_nmf, yTest_oh)
svr_acc(train_pca, yTrain_oh, test_pca, yTest_oh)

For SVR:
TrainScore:  0.8444961048851121
TestScore:  0.8420562111851597
RMSE of test:  10.25626497467541
R-square of test:  0.8420562111851597

For SVR:
TrainScore:  0.21958793703407098
TestScore:  0.22029433691451286
RMSE of test:  22.787847312912323
R-square of test:  0.22029433691451283

For SVR:
TrainScore:  0.5842962758138057
TestScore:  0.5847220029713878
RMSE of test:  16.63058082121034
R-square of test:  0.5847220029713878



In [9]:
from sklearn.tree import DecisionTreeRegressor as dtc
def dt_acc(xTrain, yTrain, xTest, yTest):
    
    ###
    param_grid = {'criterion': ['mse', 'mae'],
                 'max_depth': [x for x in range(5, 10)],
                 'min_samples_leaf': [x for x in range(2, 5)]}
    model = GridSearchCV(dtc(), param_grid, cv=5)
    model = model.fit(xTrain, yTrain['price'])
    print("For Decision Tree:")
    print("Best parameters found by grid search:")
    print(model.best_params_)
    print("Best CV score:")
    print(model.best_score_)
    
    preds = model.predict(xTest)
    error = sqrt(mean_squared_error(yTest['price'],preds))
    trainScore = model.score(xTrain, yTrain['price'])
    testScore = model.score(xTest, yTest['price'])
    r2 = r2_score(yTest['price'], preds)
    
    print()
    print('trainScore of dt: ', trainScore)
    print('testScore of dt: ', testScore)
    print('RMSE of test: ', error)
    print('R-square of test: ', r2)
    print()
    

dt_acc(xTrain_oh, yTrain_oh, xTest_oh, yTest_oh)
dt_acc(train_nmf, yTrain_oh, test_nmf, yTest_oh)
dt_acc(train_pca, yTrain_oh, test_pca, yTest_oh)

For Decision Tree:
Best parameters found by grid search:
{'criterion': 'mse', 'max_depth': 9, 'min_samples_leaf': 2}
Best CV score:
0.9156319109768469

trainScore of dt:  0.9162221805935009
testScore of dt:  0.9138067344700462
RMSE of test:  7.576604611019945
R-square of test:  0.9138067344700462

For Decision Tree:
Best parameters found by grid search:
{'criterion': 'mse', 'max_depth': 9, 'min_samples_leaf': 3}
Best CV score:
0.9363428028412129

trainScore of dt:  0.9385949550950462
testScore of dt:  0.9364925165519147
RMSE of test:  6.503548875850373
R-square of test:  0.9364925165519146

For Decision Tree:
Best parameters found by grid search:
{'criterion': 'mse', 'max_depth': 9, 'min_samples_leaf': 2}
Best CV score:
0.7982680283628397

trainScore of dt:  0.7999277060837442
testScore of dt:  0.791372823183729
RMSE of test:  11.787545907712197
R-square of test:  0.791372823183729



In [10]:
from sklearn.ensemble import RandomForestRegressor
def rf_acc(xTrain, yTrain, xTest, yTest):
    
    ###
    param_grid = {'n_estimators': [x for x in range(50, 100, 10)]}
    model = GridSearchCV(RandomForestRegressor(), param_grid, cv=5)
    model = model.fit(xTrain, yTrain['price'])
    print("For Random Forest:")
    print("Best parameters found by grid search:")
    print(model.best_params_)
    print("Best CV score:")
    print(model.best_score_)
    
    preds = model.predict(xTest)
    error = sqrt(mean_squared_error(yTest['price'],preds))
    trainScore = model.score(xTrain, yTrain['price'])
    testScore = model.score(xTest, yTest['price'])
    r2 = r2_score(yTest['price'], preds)
    
    print()
    print('trainScore of rf: ', trainScore)
    print('testScore of rf: ', testScore)
    print('RMSE of test: ', error)
    print('R-square of test: ', r2)
    print()

rf_acc(xTrain_oh, yTrain_oh, xTest_oh, yTest_oh)
rf_acc(train_nmf, yTrain_oh, test_nmf, yTest_oh)
rf_acc(train_pca, yTrain_oh, test_pca, yTest_oh)

For Random Forest:
Best parameters found by grid search:
{'n_estimators': 70}
Best CV score:
0.9759156516120948

trainScore of rf:  0.9778575198609446
testScore of rf:  0.9770438129221204
RMSE of test:  3.9100997272194573
R-square of test:  0.9770438129221205

For Random Forest:
Best parameters found by grid search:
{'n_estimators': 90}
Best CV score:
0.9757849008942758

trainScore of rf:  0.9778520693481848
testScore of rf:  0.9769499279573899
RMSE of test:  3.918087225390005
R-square of test:  0.9769499279573899

For Random Forest:
Best parameters found by grid search:
{'n_estimators': 60}
Best CV score:
0.9701444911725162

trainScore of rf:  0.9739009795053342
testScore of rf:  0.9719574208582197
RMSE of test:  4.321623552167929
R-square of test:  0.9719574208582198



In [11]:
from xgboost import XGBRegressor
def xgb_acc(xTrain, yTrain, xTest, yTest):
    
    ###
    param_grid = {'colsample_bytree':[0.4,0.6,0.8],
                  'min_child_weight':[1.5,6,10],
                  'learning_rate':[0.1,0.2,0.5],
                  'max_depth':[3,5,7],
                  'n_estimators':[20,50,80]}
    model = GridSearchCV(XGBRegressor(objective ='reg:squarederror'), param_grid, cv=5)
    model = model.fit(xTrain, yTrain['price'])
    print("For XGBoost:")
    print("Best parameters found by grid search:")
    print(model.best_params_)
    print("Best CV score:")
    print(model.best_score_)
    
    preds = model.predict(xTest)
    error = sqrt(mean_squared_error(yTest['price'],preds))
    trainScore = model.score(xTrain, yTrain['price'])
    testScore = model.score(xTest, yTest['price'])
    r2 = r2_score(yTest['price'], preds)
    
    print()
    print('trainScore of XGB: ', trainScore)
    print('testScore of XGB: ', testScore)
    print('RMSE of test: ', error)
    print('R-square of test: ', r2)
    print()

xgb_acc(xTrain_oh, yTrain_oh, xTest_oh, yTest_oh)
xgb_acc(train_nmf, yTrain_oh, test_nmf, yTest_oh)
xgb_acc(train_pca, yTrain_oh, test_pca, yTest_oh)

For XGBoost:
Best parameters found by grid search:
{'colsample_bytree': 0.8, 'learning_rate': 0.5, 'max_depth': 7, 'min_child_weight': 1.5, 'n_estimators': 80}
Best CV score:
0.9746529564773688

trainScore of XGB:  0.9770405145176104
testScore of XGB:  0.9759909195804495
RMSE of test:  3.9987635040580387
R-square of test:  0.9759909195804495

For XGBoost:
Best parameters found by grid search:
{'colsample_bytree': 0.6, 'learning_rate': 0.5, 'max_depth': 7, 'min_child_weight': 1.5, 'n_estimators': 80}
Best CV score:
0.9755940189952952

trainScore of XGB:  0.977724830260049
testScore of XGB:  0.976854255322022
RMSE of test:  3.926210097280589
R-square of test:  0.976854255322022

For XGBoost:
Best parameters found by grid search:
{'colsample_bytree': 0.4, 'learning_rate': 0.5, 'max_depth': 7, 'min_child_weight': 1.5, 'n_estimators': 80}
Best CV score:
0.9236491016527134

trainScore of XGB:  0.9390318634756151
testScore of XGB:  0.9347602226227991
RMSE of test:  6.5916507955828045
R-square